# Homework 4
In this homework you will train your first convolutional neural network on  images from supertux. 

Development notes: 

1) If you are doing your homework in a Jupyter/iPython notebook you may need to 'Restart & Clear Output' after making a change and re-running a cell.  TensorFlow will not allow you to create multiple variables with the same name, which is what you are doing when you run a cell that creates a variable twice.<br/><br/>
2) Be careful with your calls to global_variables_initializer(). If you call it after training one network it will re-initialize your variables erasing your training.  In general, double check the outputs of your model after all training and before turning your model in. Ending a session will discard all your variable values.

## Part 0: Setup

In [1]:
import tensorflow as tf
import numpy as np
import util

# Load the data we are giving you
def load(filename, W=64, H=64):
    data = np.fromfile(filename, dtype=np.uint8).reshape((-1, W*H*3+1))
    images, labels = data[:, :-1].reshape((-1,H,W,3)), data[:, -1]
    return images, labels

image_data, label_data = load('tux_train.dat')

print('Input shape: ' + str(image_data.shape))
print('Labels shape: ' + str(label_data.shape))

num_classes = 6

Input shape: (12257, 64, 64, 3)
Labels shape: (12257,)


## Part 1: Define your convnet

Make sure the total number of parameters is less than 100,000.

In [12]:
# IMPORTANT:
# Give each of your layers a scope=(<layer_name>) argument
#    with layer_name as 'conv1'...'conv5' or 'pool'

# Lets clear the tensorflow graph, so that you don't have to restart the notebook every time you change the network
tf.reset_default_graph()

# Set up your input placeholder
inputs = tf.placeholder(tf.float32, (None,64,64,3), name='input')

# Whenever you deal with image data it's important to mean center it first and subtract the standard deviation
white_inputs = (inputs - 100.) / 72.

# Set up your label placeholders
labels = tf.placeholder(tf.int64, (None), name='labels')

# Step 1: define the compute graph of your CNN here
#   Use 5 conv2d layers (tf.contrib.layers.conv2d) and one pooling layer tf.contrib.layers.max_pool2d or tf.contrib.layers.avg_pool2d.
#   The output of the network should be a None x 1 x 1 x 6 tensor.
#   Make sure the last conv2d does not have a ReLU: activation_fn=None

conv1 = tf.contrib.layers.conv2d(
  inputs=white_inputs,
  num_outputs=16,
  kernel_size=[3, 3],
  stride=2,
  padding="same",
  scope='conv1') # put in scope everywhere
print(conv1)

# Pooling Layer #1
pool1 = tf.contrib.layers.max_pool2d(inputs=conv1, kernel_size=[3,3], stride=2, scope='pool1')
print(pool1)

# Convolutional Layer #2
conv2 = tf.contrib.layers.conv2d(
    inputs=pool1,
    num_outputs=32,
    kernel_size=[3, 3],
    stride=2,
    padding="same",
    scope='conv2')
print(conv2)

# Convolutional Layer #2
conv3 = tf.contrib.layers.conv2d(
    inputs=conv2,
    num_outputs=64,
    kernel_size=[3, 3],
    stride=2,
    padding="same",
    scope='conv3')
print(conv3)

conv4 = tf.contrib.layers.conv2d(
    inputs=conv3,
    num_outputs=120,
    kernel_size=[3, 3],
    stride=2,
    padding="same",
    scope='conv4')
print(conv4)

conv5 = tf.contrib.layers.conv2d(
    inputs=conv4,
    num_outputs=6,
    kernel_size=[3, 3],
    activation_fn=None,
    stride=2,
    padding="same", 
    scope='conv5')
print(conv5)

h= tf.identity(conv5, name='conv_out')

# # The input here should be a   None x 1 x 1 x 6   tensor
output = tf.identity(tf.contrib.layers.flatten(h), name='output')

# # Step 2: use a classification loss function (from assignment 3)
loss =  tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=output))
# # Step 3: create an optimizer (from assignment 3)
optimizer = tf.train.AdamOptimizer(0.0001,0.9,0.999)
# # Step 4: use that optimizer on your loss function (from assignment 3)
minimizer = optimizer.minimize(loss)
# # Step 5: calculate some metrics
correct = tf.equal(tf.argmax(output, 1), labels)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]), '/', 100000 )

Tensor("conv1/Relu:0", shape=(?, 32, 32, 16), dtype=float32)
Tensor("pool1/MaxPool:0", shape=(?, 15, 15, 16), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 8, 8, 32), dtype=float32)
Tensor("conv3/Relu:0", shape=(?, 4, 4, 64), dtype=float32)
Tensor("conv4/Relu:0", shape=(?, 2, 2, 120), dtype=float32)
Tensor("conv5/BiasAdd:0", shape=(?, 1, 1, 6), dtype=float32)
Total number of variables used  99310 / 100000


## Part 2: Training

Training might take up to 20 min depending on your architecture.  This time around you should get close to 100% trianing accuracy.

In [7]:
def train_conv(batch_images,batch_labels):
    _,acc_tc, loss_tc = sess.run([minimizer,accuracy,loss],feed_dict={inputs:batch_images,labels: batch_labels})
    return acc_tc,loss_tc

In [8]:
# Batch size
BS = 32

# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())

# This is a helper function that trains your model for several epochs un shuffled data
# train_func should take a single step in the optmimzation and return accuracy and loss
#   accuracy, loss = train_func(batch_images, batch_labels)
# HINT: train_func should call sess.run
def train(train_func):
    # An epoch is a single pass over the training data
    for epoch in range(20):
        # Let's shuffle the data every epoch
        np.random.seed(epoch)
        np.random.shuffle(image_data)
        np.random.seed(epoch)
        np.random.shuffle(label_data)
        # Go through the entire dataset once
        accs, losss = [], []
        for i in range(0, image_data.shape[0]-BS+1, BS):
            # Train a single batch
            batch_images, batch_labels = image_data[i:i+BS], label_data[i:i+BS]
            acc, loss = train_func(batch_images, batch_labels)
            accs.append(acc)
            losss.append(loss)
        print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f'%(epoch, np.mean(accs), np.mean(losss)))


# Train convnet
print('Convnet')
### Your Code Here (from assignment 3) ### 
train(train_conv)

Convnet
[  0] Accuracy: 0.747  	  Loss: 0.786
[  1] Accuracy: 0.904  	  Loss: 0.302
[  2] Accuracy: 0.923  	  Loss: 0.235
[  3] Accuracy: 0.935  	  Loss: 0.200
[  4] Accuracy: 0.942  	  Loss: 0.173
[  5] Accuracy: 0.948  	  Loss: 0.155
[  6] Accuracy: 0.954  	  Loss: 0.139
[  7] Accuracy: 0.957  	  Loss: 0.130
[  8] Accuracy: 0.961  	  Loss: 0.116
[  9] Accuracy: 0.965  	  Loss: 0.107
[ 10] Accuracy: 0.967  	  Loss: 0.098
[ 11] Accuracy: 0.969  	  Loss: 0.091
[ 12] Accuracy: 0.973  	  Loss: 0.083
[ 13] Accuracy: 0.974  	  Loss: 0.077
[ 14] Accuracy: 0.978  	  Loss: 0.071
[ 15] Accuracy: 0.979  	  Loss: 0.066
[ 16] Accuracy: 0.981  	  Loss: 0.061
[ 17] Accuracy: 0.983  	  Loss: 0.055
[ 18] Accuracy: 0.984  	  Loss: 0.051
[ 19] Accuracy: 0.984  	  Loss: 0.050


## Part 3: Evaluation

### See your model

In [9]:
# Show the current graph
util.show_graph(tf.get_default_graph().as_graph_def())

### Compute the valiation accuracy
The convnet still massively overfits. We will deal with this in assignment 5.

In [10]:
image_val, label_val = load('tux_val.dat')

print('Input shape: ' + str(image_val.shape))
print('Labels shape: ' + str(label_val.shape))

val_accuracy, val_loss = sess.run([accuracy, loss], feed_dict={inputs: image_val, labels: label_val})
print("ConvNet Validation Accuracy: ", val_accuracy)

Input shape: (3912, 64, 64, 3)
Labels shape: (3912,)
ConvNet Validation Accuracy:  0.944785


## Part 4: Save Model
Please note that we also want you to turn in your ipynb for this assignment.  Zip up the ipynb along with the tfg for your submission.

In [11]:
util.save('assignment4.tfg', session=sess)